<a href="https://colab.research.google.com/github/RebecaGis/Py/blob/main/Geocoding_para_Features_Selecionadas_em_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arcpy
import requests
import time

# Configurações
shapefile = r"C:CAMINHO/DO/ARQUIVO/AQUI"
campo_lat = "Latitude"  # substitua pelo nome real do campo
campo_lon = "Longitude"  # substitua pelo nome real do campo
campo_endereco = "Endereço"  # campo a ser preenchido

# Verificar se há features selecionadas
num_selecionados = int(arcpy.GetCount_management(shapefile).getOutput(0))
if num_selecionados == 0:
    print("Nenhuma feature selecionada. Por favor, selecione pelo menos uma feature para geocodificar.")
    exit()

# Verificar se o campo Endereço existe, se não, criar
if campo_endereco not in [field.name for field in arcpy.ListFields(shapefile)]:
    arcpy.AddField_management(shapefile, campo_endereco, "TEXT", field_length=200)

# Função para geocodificação reversa usando Nominatim (OpenStreetMap)
def get_address(lat, lon):
    try:
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=18&addressdetails=1"
        headers = {'User-Agent': 'PDAU-Geocoding/1.0'}  # Necessário para Nominatim
        response = requests.get(url, headers=headers).json()

        # Extrair componentes do endereço
        address = response.get('address', {})
        endereco = ""

        # Construir endereço formatado
        if 'road' in address:
            endereco += address['road']
            if 'house_number' in address:
                endereco += f", {address['house_number']}"
        elif 'pedestrian' in address:
            endereco += address['pedestrian']

        if 'suburb' in address:
            endereco += f", {address['suburb']}"
        elif 'neighbourhood' in address:
            endereco += f", {address['neighbourhood']}"

        if 'city' in address:
            endereco += f", {address['city']}"
        elif 'town' in address:
            endereco += f", {address['town']}"

        if 'state' in address:
            endereco += f", {address['state']}"
        if 'postcode' in address:
            endereco += f", {address['postcode']}"
        if 'country' in address:
            endereco += f", {address['country']}"

        return endereco if endereco else response.get('display_name', 'Endereço não encontrado')
    except Exception as e:
        print(f"Erro ao geocodificar: {str(e)}")
        return "Erro na geocodificação"

# Atualizar apenas as features selecionadas
with arcpy.da.UpdateCursor(shapefile, [campo_lat, campo_lon, campo_endereco], where_clause=None, sql_clause=(None, 'ORDER BY OBJECTID')) as cursor:
    contador = 0
    total = num_selecionados
    for row in cursor:
        if not row[2] or row[2] in ("", "Endereço não encontrado", "Erro na geocodificação"):
            lat, lon = row[0], row[1]
            if lat and lon:
                print(f"Processando {contador+1} de {total}...")
                endereco = get_address(lat, lon)
                row[2] = endereco
                cursor.updateRow(row)
                contador += 1
                time.sleep(1.2)  # Respeitar limite de 1 requisição por segundo do Nominatim
            else:
                row[2] = "Coordenadas ausentes"
                cursor.updateRow(row)

print(f"\nProcesso concluído! {contador} endereços foram geocodificados.")